In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
from UnarySim.sw.kernel.linear import UnaryLinear
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressivePrecision


In [2]:
import os
cwd = "/Users/wudidaizi/Project/stochasticSim/pytorchSim"
print(cwd)
model_path = cwd+"/saved_model_state_dict_8"
print(model_path)

/Users/wudidaizi/Project/stochasticSim/pytorchSim
/Users/wudidaizi/Project/stochasticSim/pytorchSim/saved_model_state_dict_8


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# MNIST data loader
transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root=cwd+'/data/mnist', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)

testset = torchvision.datasets.MNIST(root=cwd+'/data/mnist', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, num_workers=4)

In [5]:
 class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        x = torch.clamp(x, -1, 1)
        x = F.avg_pool2d(x, (2, 2))
        x = F.relu(x)
        # If the size is a square you can only specify a single number
        x = self.conv2(x)
        x = torch.clamp(x, -1, 1)
        x = F.avg_pool2d(x, (2, 2))
        x = F.relu(x)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = torch.clamp(x, -1, 1)
        x = F.relu(self.fc2(x))
        x = torch.clamp(x, -1, 1)
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc1_drop = nn.Dropout(0.6)
        self.fc2 = nn.Linear(512, 256)
        self.fc2_drop = nn.Dropout(0.6)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)        
        x = torch.clamp(x, -1, 1)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        x = torch.clamp(x, -1, 1)
        return F.log_softmax(self.fc3(x), dim=1)
    
model = Net()
model.to(device)


Net(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc1_drop): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc2_drop): Dropout(p=0.6, inplace=False)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

Net(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc1_drop): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc2_drop): Dropout(p=0.6, inplace=False)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [7]:
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data[0].to(device), data[1].to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#     print(images.size())
# print('Accuracy of the network on the 10000 test images: %f %%' % (
#     100 * correct / total))

In [8]:
fc = nn.Linear(1024, 512)
fc.weight = model.fc1.weight
fc.bias = model.fc1.bias
ufc = UnaryLinear(1024, 512, 256, model.fc1.weight, model.fc1.bias)
print(fc.weight.size())

torch.Size([512, 1024])


In [9]:
iVec = (((torch.rand(1, 1024) * 2 - 1)/4)*256).floor()/256
# print(inVec)
oVec = fc(iVec)
oVec = oVec.clamp_(-1.,1.).mul_(256).floor()/256
# print(inVec)
print(oVec)

tensor([[-0.2969,  0.9102, -0.4883, -0.9414, -0.3828, -0.3711, -0.3086,  0.7148,
         -0.4961, -1.0000, -1.0000, -0.2109,  0.3359,  1.0000, -0.0195,  0.0664,
          0.2852,  0.5312,  1.0000,  0.6914, -0.6523, -0.4883,  0.0391, -0.1133,
          0.4844, -0.1758, -1.0000,  0.1758, -0.2578,  0.0664, -0.9297, -0.7383,
          0.6055,  0.0781, -0.9648, -1.0000,  0.0586,  0.5977, -0.6328,  0.2031,
         -0.2070, -1.0000, -0.9102,  0.6875,  0.6094,  0.5117, -1.0000, -0.0547,
          0.7383,  0.4492, -0.7539, -1.0000, -0.1250,  0.7656,  1.0000,  0.0742,
         -0.2461, -1.0000,  0.4141, -0.1016, -0.0703, -0.2109,  0.2969,  0.8633,
          1.0000,  0.6523, -1.0000,  0.5000, -0.1445,  1.0000, -0.3203, -0.0859,
          0.7891, -1.0000,  1.0000,  0.0234,  0.2812,  0.2227, -1.0000, -0.9609,
         -0.4648, -0.6914, -0.4648,  0.3828,  1.0000, -1.0000,  0.3008, -0.1289,
         -1.0000,  0.4727,  0.2188, -0.6211, -0.2812, -1.0000, -0.1406, -0.3047,
         -1.0000, -1.0000, -

In [10]:
iVecSource = SourceGen(iVec, bitwidth=8, mode="bipolar").Gen()
iVecRNG = RNG(8, 1, "Sobol").Out()
iVecBS = BSGen(iVecSource, iVecRNG)

iVecPP = ProgressivePrecision(iVec, mode="bipolar")
oVecPP = ProgressivePrecision(oVec, mode="bipolar")

with torch.no_grad():
    idx = torch.zeros(iVecSource.size()).type(torch.long)
    for i in range(256):
        iBS = iVecBS.Gen(idx + i)
        iVecPP.Monitor(iBS)
        oVecU = ufc(iBS)
#         oVecPP.Monitor(oVec)
    print("input error: ", min(min(iVecPP.Report())), max(max(iVecPP.Report())))
#     print("output error:", min(min(oVecPP.Report())), max(max(oVecPP.Report())))
    outProb = (oVecU-1024*128)/256
    outProb.clamp_(-1.,1.)
    print("output count:", outProb - oVec)


input error:  tensor(-0.0039) tensor(0.0039)
output count: tensor([[ 0.6875, -0.0273,  0.6680,  0.9883,  0.6445,  0.6680,  0.6875,  0.1797,
          0.8086,  0.4180,  0.6953,  0.6250,  0.3008,  0.0000,  0.4570,  0.5391,
          0.4141,  0.2383,  0.0000,  0.2578,  0.7812,  0.8125,  0.4180,  0.5586,
          0.4336,  0.6367,  0.8086,  0.5352,  0.5039,  0.6406,  0.9883,  0.8867,
          0.0938,  0.5117,  0.9727,  0.7695,  0.5430,  0.1992,  0.8867,  0.4258,
          0.6328,  0.8867,  1.0586,  0.1914,  0.2344,  0.3945,  0.7109,  0.5547,
          0.1797,  0.4102,  0.9219,  0.6719,  0.5977,  0.2305,  0.0000,  0.4648,
          0.7266,  0.2227,  0.3984,  0.5938,  0.5078,  0.6836,  0.4062,  0.0273,
          0.0000,  0.2969,  0.6992,  0.3320,  0.4453,  0.0000,  0.7773,  0.5742,
          0.1367,  0.4727,  0.0000,  0.6250,  0.4180,  0.3633,  0.7188,  1.0195,
          0.8203,  0.9023,  0.7852,  0.2969,  0.0000,  0.6328,  0.3750,  0.7422,
          0.2148,  0.3281,  0.5273,  0.7969,  0.77